# Synthetic Database Log Generation
This section defines the database schema, user roles, and functions to generate synthetic database access logs, including both normal and anomalous queries. The generated logs are saved to a JSON file for further analysis.

In [497]:
import random
from datetime import datetime, timedelta
import json

# Database schema with relationships
tables = {
    'patients': {
        'columns': ['patient_id', 'first_name', 'last_name', 'dob', 'gender', 'ssn', 'address', 'phone'],
        'pk': 'patient_id',
        'relationships': {
            'visits': 'patient_id',
            'billing': 'patient_id'
        }
    },
    'visits': {
        'columns': ['visit_id', 'patient_id', 'doctor_id', 'visit_date', 'diagnosis', 'treatment'],
        'pk': 'visit_id',
        'fk': {'patient_id': 'patients', 'doctor_id': 'employees'},
        'relationships': {
            'billing': 'visit_id'
        }
    },
    'billing': {
        'columns': ['bill_id', 'visit_id', 'patient_id', 'amount', 'status', 'insurance_info'],
        'pk': 'bill_id',
        'fk': {'visit_id': 'visits', 'patient_id': 'patients'}
    },
    'employees': {
        'columns': ['employee_id', 'first_name', 'last_name', 'role', 'department', 'salary'],
        'pk': 'employee_id',
        'relationships': {
            'visits': 'doctor_id'
        }
    }
}

# Standardized role definitions with query frequencies
roles = {
    'doctor': {
        'query_frequency': {'SELECT': 0.8, 'UPDATE': 0.2},
        'allowed_joins': [
            ['patients', 'visits'],
            ['visits', 'billing']
        ],
        'access': {
            'patients': {
                'columns': ['patient_id', 'first_name', 'last_name', 'dob', 'gender'],
                'operations': ['SELECT'],
                'join_condition': "patients.patient_id = visits.patient_id AND visits.doctor_id = 'current_user'"
            },
            'visits': {
                'columns': '*',
                'operations': ['SELECT', 'UPDATE'],
                'condition': "doctor_id = 'current_user'"
            },
            'billing': {
                'columns': ['bill_id', 'visit_id', 'status'],
                'operations': ['SELECT'],
                'join_condition': "visits.visit_id = billing.visit_id AND visits.doctor_id = 'current_user'"
            }
        }
    },
    'billing_clerk': {
        'query_frequency': {'SELECT': 0.6, 'INSERT': 0.2, 'UPDATE': 0.2},
        'allowed_joins': [
            ['patients', 'billing'],
            ['visits', 'billing']
        ],
        'access': {
            'patients': {
                'columns': ['patient_id', 'first_name', 'last_name', 'address', 'phone'],
                'operations': ['SELECT']
            },
            'visits': {
                'columns': ['visit_id', 'patient_id', 'visit_date'],
                'operations': ['SELECT']
            },
            'billing': {
                'columns': '*',
                'operations': ['SELECT', 'INSERT', 'UPDATE']
            }
        }
    },
    'admin': {
        'query_frequency': {'SELECT': 0.5, 'INSERT': 0.2, 'UPDATE': 0.2, 'DELETE': 0.1},
        'allowed_joins': [
            ['patients', 'visits'],
            ['patients', 'billing'],
            ['visits', 'billing'],
            ['visits', 'employees']
        ],
        'access': {
            'patients': {'columns': '*', 'operations': ['SELECT', 'INSERT', 'UPDATE', 'DELETE']},
            'visits': {'columns': '*', 'operations': ['SELECT', 'INSERT', 'UPDATE', 'DELETE']},
            'billing': {'columns': '*', 'operations': ['SELECT', 'INSERT', 'UPDATE', 'DELETE']},
            'employees': {'columns': '*', 'operations': ['SELECT', 'INSERT', 'UPDATE', 'DELETE']}
        }
    }
}

def generate_phone_number():
    """Generate a random phone number without Faker"""
    return f"{random.randint(200, 999)}-{random.randint(200, 999)}-{random.randint(1000, 9999)}"

def generate_ssn():
    """Generate a random SSN without Faker"""
    return f"{random.randint(100, 999)}-{random.randint(10, 99)}-{random.randint(1000, 9999)}"

def generate_name():
    """Generate a random name without Faker"""
    first_names = ['John', 'Jane', 'Robert', 'Emily', 'Michael', 'Sarah', 'David', 'Lisa']
    last_names = ['Smith', 'Johnson', 'Williams', 'Brown', 'Jones', 'Miller', 'Davis']
    return random.choice(first_names), random.choice(last_names)

def generate_realistic_value(column_name):
    """Generate realistic values based on column names without Faker"""
    if column_name.endswith('_id'):
        return str(random.randint(1000, 9999))
    elif column_name == 'first_name':
        return generate_name()[0]
    elif column_name == 'last_name':
        return generate_name()[1]
    elif column_name == 'dob':
        year = random.randint(1940, 2005)
        month = random.randint(1, 12)
        day = random.randint(1, 28)
        return f"{year}-{month:02d}-{day:02d}"
    elif column_name == 'gender':
        return random.choice(['M', 'F', 'O'])
    elif column_name == 'ssn':
        return generate_ssn()
    elif column_name == 'address':
        street = random.choice(['Main', 'Oak', 'Pine', 'Maple', 'Cedar'])
        return f"{random.randint(1, 999)} {street} St"
    elif column_name == 'phone':
        return generate_phone_number()
    elif column_name.endswith('_date'):
        year = random.randint(2020, 2023)
        month = random.randint(1, 12)
        day = random.randint(1, 28)
        return f"{year}-{month:02d}-{day:02d}"
    elif column_name == 'amount':
        return round(random.uniform(50, 5000), 2)
    elif column_name == 'salary':
        return round(random.uniform(30000, 150000), 2)
    elif column_name in ['diagnosis', 'treatment', 'insurance_info', 'department']:
        return random.choice(['Standard', 'Premium', 'Emergency', 'Routine', 'Special'])
    elif column_name == 'status':
        return random.choice(['Pending', 'Paid', 'Denied', 'Processing'])
    else:
        return 'SampleValue'

def generate_normal_query(role):
    """Generate a normal single-table query"""
    role_config = roles[role]
    table = random.choice(list(role_config['access'].keys()))
    table_config = role_config['access'][table]

    # Get operation based on frequency
    operation = random.choices(
        list(role_config['query_frequency'].keys()),
        weights=list(role_config['query_frequency'].values()),
        k=1
    )[0]

    # Ensure operation is allowed for this table
    while operation not in table_config['operations']:
        operation = random.choices(
            list(role_config['query_frequency'].keys()),
            weights=list(role_config['query_frequency'].values()),
            k=1
        )[0]

    # Get columns to include
    if table_config['columns'] == '*':
        columns = tables[table]['columns']
    else:
        columns = table_config['columns']

    num_columns = random.randint(1, len(columns))
    selected_columns = random.sample(columns, num_columns)

    # Generate the query
    if operation == 'SELECT':
        query = f"SELECT {', '.join(selected_columns)} FROM {table}"
        if 'condition' in table_config:
            query += f" WHERE {table_config['condition']}"
        elif random.random() > 0.5:
            pk = tables[table]['pk']
            query += f" WHERE {pk} = '{generate_realistic_value(pk)}'"

    elif operation == 'INSERT':
        query = f"INSERT INTO {table} ({', '.join(selected_columns)}) VALUES ("
        values = [f"'{generate_realistic_value(col)}'" for col in selected_columns]
        query += ', '.join(values) + ")"

    elif operation == 'UPDATE':
        query = f"UPDATE {table} SET "
        updates = []
        # Don't update primary key
        updatable_columns = [col for col in selected_columns if col != tables[table]['pk']]
        if not updatable_columns:
            updatable_columns = [col for col in columns if col != tables[table]['pk']][:1]

        for col in updatable_columns:
            updates.append(f"{col} = '{generate_realistic_value(col)}'")
        query += ', '.join(updates)

        # Add WHERE clause
        if 'condition' in table_config:
            query += f" WHERE {table_config['condition']}"
        else:
            pk = tables[table]['pk']
            query += f" WHERE {pk} = '{generate_realistic_value(pk)}'"

    elif operation == 'DELETE':
        query = f"DELETE FROM {table}"
        if 'condition' in table_config:
            query += f" WHERE {table_config['condition']}"
        else:
            pk = tables[table]['pk']
            query += f" WHERE {pk} = '{generate_realistic_value(pk)}'"

    return query, operation, table, selected_columns

def generate_join_query(role, tables_to_join):
    """Generate a JOIN query between tables with proper conditions"""
    role_config = roles[role]

    # Validate the join is allowed for this role
    if tables_to_join not in role_config['allowed_joins']:
        tables_to_join = random.choice(role_config['allowed_joins'])

    table1, table2 = tables_to_join
    join_condition = None

    # Find the join condition
    if table2 in tables[table1]['relationships']:
        join_column = tables[table1]['relationships'][table2]
        join_condition = f"{table1}.{join_column} = {table2}.{join_column}"
    elif table1 in tables[table2]['relationships']:
        join_column = tables[table2]['relationships'][table1]
        join_condition = f"{table2}.{join_column} = {table1}.{join_column}"
    else:
        # If no direct relationship, find through foreign keys
        for col in tables[table1]['columns']:
            if col.endswith('_id') and col in tables[table2]['columns']:
                join_condition = f"{table1}.{col} = {table2}.{col}"
                break

    if not join_condition:
        join_condition = f"{table1}.{tables[table1]['pk']} = {table2}.{tables[table1]['pk']}"

    # Get columns for each table
    cols1 = []
    cols2 = []
    if role_config['access'][table1]['columns'] == '*':
        cols1 = tables[table1]['columns']
    else:
        cols1 = random.sample(role_config['access'][table1]['columns'],
                            random.randint(1, len(role_config['access'][table1]['columns'])))

    if role_config['access'][table2]['columns'] == '*':
        cols2 = tables[table2]['columns']
    else:
        cols2 = random.sample(role_config['access'][table2]['columns'],
                            random.randint(1, len(role_config['access'][table2]['columns'])))

    # Add table prefixes to columns
    cols1 = [f"{table1}.{col}" for col in cols1]
    cols2 = [f"{table2}.{col}" for col in cols2]

    # Generate the query
    query = f"SELECT {', '.join(cols1 + cols2)} FROM {table1} JOIN {table2} ON {join_condition}"

    # Add role-specific conditions
    conditions = []
    if 'condition' in role_config['access'][table1]:
        conditions.append(role_config['access'][table1]['condition'])
    if 'condition' in role_config['access'][table2]:
        conditions.append(role_config['access'][table2]['condition'])
    if 'join_condition' in role_config['access'][table1]:
        conditions.append(role_config['access'][table1]['join_condition'])
    if 'join_condition' in role_config['access'][table2]:
        conditions.append(role_config['access'][table2]['join_condition'])

    if conditions:
        query += " WHERE " + " AND ".join(conditions)
    elif random.random() > 0.5:
        # Add a random condition
        pk = tables[table1]['pk']
        query += f" WHERE {table1}.{pk} = '{generate_realistic_value(pk)}'"

    return query, 'SELECT', [table1, table2], cols1 + cols2

def generate_anomalous_query(role):
    """Generate an anomalous single-table query"""
    role_config = roles[role]
    table = random.choice(list(role_config['access'].keys()))
    table_config = role_config['access'][table]

    # Choose an anomalous pattern
    anomaly_type = random.choice([
        'unauthorized_operation',
        'unauthorized_columns',
        'excessive_data_access'
    ])

    if anomaly_type == 'unauthorized_operation':
        # Use operation not in allowed operations
        allowed_ops = table_config['operations']
        all_ops = ['SELECT', 'INSERT', 'UPDATE', 'DELETE']
        unauthorized_ops = [op for op in all_ops if op not in allowed_ops]
        if not unauthorized_ops:
            return generate_normal_query(role)  # Fallback if no unauthorized operations
        operation = random.choice(unauthorized_ops)
    else:
        operation = random.choice(table_config['operations'])

    if anomaly_type == 'unauthorized_columns':
        # Access columns not normally allowed
        if table_config['columns'] == '*':
            columns = tables[table]['columns']
        else:
            all_columns = tables[table]['columns']
            unauthorized_columns = [col for col in all_columns if col not in table_config['columns']]
            if not unauthorized_columns:
                return generate_normal_query(role)  # Fallback if no unauthorized columns
            columns = random.sample(unauthorized_columns, min(2, len(unauthorized_columns)))
            # Mix with some authorized columns
            if random.random() > 0.3 and table_config['columns']:
                columns.extend(random.sample(table_config['columns'], 1))
    elif anomaly_type == 'excessive_data_access':
        # Access all columns
        columns = tables[table]['columns']
    else:
        # Normal column selection
        if table_config['columns'] == '*':
            columns = tables[table]['columns']
        else:
            columns = table_config['columns']
        columns = random.sample(columns, random.randint(1, len(columns)))

    # Generate the query
    if operation == 'SELECT':
        query = f"SELECT {', '.join(columns)} FROM {table}"
        if anomaly_type == 'excessive_data_access' and random.random() > 0.5:
            query += " WHERE 1=1"  # Dummy condition to select all rows
        elif 'condition' in table_config:
            query += f" WHERE {table_config['condition']}"
        elif random.random() > 0.5:
            pk = tables[table]['pk']
            query += f" WHERE {pk} = '{generate_realistic_value(pk)}'"

    elif operation == 'INSERT':
        query = f"INSERT INTO {table} ({', '.join(columns)}) VALUES ("
        values = [f"'{generate_realistic_value(col)}'" for col in columns]
        query += ', '.join(values) + ")"

    elif operation == 'UPDATE':
        query = f"UPDATE {table} SET "
        updates = [f"{col} = '{generate_realistic_value(col)}'" for col in columns
                  if col != tables[table]['pk']]
        query += ', '.join(updates)
        pk = tables[table]['pk']
        query += f" WHERE {pk} = '{generate_realistic_value(pk)}'"

    elif operation == 'DELETE':
        query = f"DELETE FROM {table}"
        pk = tables[table]['pk']
        query += f" WHERE {pk} = '{generate_realistic_value(pk)}'"

    return query, operation, table, columns

def generate_anomalous_join_query(role):
    """Generate an anomalous JOIN query"""
    role_config = roles[role]

    # Choose an anomalous pattern
    anomaly_type = random.choice([
        'unauthorized_join',
        'excessive_join',
        'join_with_unauthorized_columns'
    ])

    if anomaly_type == 'unauthorized_join':
        # Join tables not normally allowed
        all_possible_joins = [
            ['patients', 'visits'],
            ['patients', 'billing'],
            ['visits', 'billing'],
            ['visits', 'employees']
        ]
        unauthorized_joins = [j for j in all_possible_joins if j not in role_config['allowed_joins']]
        if not unauthorized_joins:
            unauthorized_joins = all_possible_joins  # Fallback if all joins are technically allowed

        tables_to_join = random.choice(unauthorized_joins)
        return generate_join_query(role, tables_to_join)

    elif anomaly_type == 'excessive_join':
        # Join too many tables (even if individually allowed)
        tables_to_join = random.sample(list(tables.keys()), 3)
        table1, table2, table3 = tables_to_join

        # Generate multiple joins
        query = f"SELECT * FROM {table1} JOIN {table2} ON {table1}.patient_id = {table2}.patient_id "
        query += f"JOIN {table3} ON {table2}.visit_id = {table3}.visit_id"
        return query, 'SELECT', tables_to_join, ['*']

    elif anomaly_type == 'join_with_unauthorized_columns':
        # Use allowed join but include unauthorized columns
        tables_to_join = random.choice(role_config['allowed_joins'])
        table1, table2 = tables_to_join

        # Get all columns including unauthorized ones
        cols1 = tables[table1]['columns']
        cols2 = tables[table2]['columns']

        # Add table prefixes
        cols1 = [f"{table1}.{col}" for col in cols1]
        cols2 = [f"{table2}.{col}" for col in cols2]

        # Find join condition
        join_condition = f"{table1}.patient_id = {table2}.patient_id"  # Simplified

        query = f"SELECT {', '.join(cols1 + cols2)} FROM {table1} JOIN {table2} ON {join_condition}"
        return query, 'SELECT', tables_to_join, cols1 + cols2

def generate_log_entry(is_normal=True):
    """Generate a complete log entry"""
    role = random.choice(list(roles.keys()))
    user_id = f"{role[0]}{random.randint(100, 999)}"
    timestamp = datetime.now() - timedelta(
        days=random.randint(0, 365),
        hours=random.randint(0, 23),
        minutes=random.randint(0, 59))

    # Decide whether to generate a join query
    if (is_normal and random.random() < 0.3) or (not is_normal and random.random() < 0.5):
        if is_normal:
            # Normal join from allowed joins
            tables_to_join = random.choice(roles[role]['allowed_joins'])
            query, operation, tables_accessed, columns = generate_join_query(role, tables_to_join)
        else:
            # Anomalous join
            query, operation, tables_accessed, columns = generate_anomalous_join_query(role)
    else:
        # Single table query
        if is_normal:
            query, operation, table, columns = generate_normal_query(role)
            tables_accessed = [table]
        else:
            query, operation, table, columns = generate_anomalous_query(role)
            tables_accessed = [table]

    return {
        'timestamp': timestamp.isoformat(),
        'user_id': user_id,
        'role': role,
        'query': query,
        'operation': operation,
        'tables': tables_accessed,
        'columns': columns,
        'is_normal': is_normal,
        'is_join': len(tables_accessed) > 1
    }

def generate_logs(num_normal=900, num_anomalous=100, output_file='database_logs.json'):
    """Generate a set of logs and save to file"""
    logs = []

    # Generate normal logs
    for _ in range(num_normal):
        logs.append(generate_log_entry(is_normal=True))

    # Generate anomalous logs
    for _ in range(num_anomalous):
        logs.append(generate_log_entry(is_normal=False))

    # Shuffle the logs
    random.shuffle(logs)

    # Save to file
    with open(output_file, 'w') as f:
        json.dump(logs, f, indent=2)

    print(f"Generated {len(logs)} log entries ({num_normal} normal, {num_anomalous} anomalous)")
    print(f"Saved to {output_file}")

# Generate logs
generate_logs()

Generated 1000 log entries (900 normal, 100 anomalous)
Saved to database_logs.json


# Feature Extraction Setup
Import libraries and set up the environment for extracting features from the generated database logs.

In [498]:
import numpy as np
import pandas as pd
import re
from collections import defaultdict
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split

# Load Generated Logs
Read the generated database logs from the JSON file into a pandas DataFrame for analysis.

In [499]:
df=pd.read_json('database_logs.json')
df.head()

,timestamp,user_id,role,query,operation,tables,columns,is_normal,is_join
0,2025-02-24 23:29:13.699838,b249,billing_clerk,"SELECT insurance_info, status, visit_id, amoun...",SELECT,[billing],"[insurance_info, status, visit_id, amount]",True,False
1,2024-08-15 22:08:13.728513,d151,doctor,SELECT patient_id FROM visits WHERE doctor_id ...,SELECT,[visits],[patient_id],True,False
2,2025-02-02 03:34:13.686448,a961,admin,"SELECT department, role, last_name, employee_i...",SELECT,[employees],"[department, role, last_name, employee_id]",True,False
3,2024-08-21 23:53:13.720165,d154,doctor,"SELECT visits.visit_id, visits.patient_id, vis...",SELECT,"[visits, billing]","[visits.visit_id, visits.patient_id, visits.do...",True,True
4,2025-03-10 05:11:13.720634,a480,admin,"SELECT phone, ssn FROM patients WHERE patient_...",SELECT,[patients],"[phone, ssn]",True,False


# Prepare Features and Labels
Separate the features and labels for machine learning tasks.

In [500]:
X=df.drop('role',axis=1)
y=df['role']

# Explore Unique Roles
Display the unique user roles present in the dataset.

In [501]:
df['role'].unique()

array(['billing_clerk', 'doctor', 'admin'], dtype=object)

# SQL Query Parsing
Extract tables and columns accessed in each SQL query using regular expressions.

In [502]:
import re
from collections import defaultdict

def extract_tables_and_columns(sql):
    table_matches = re.findall(r'(?:FROM|JOIN)\s+(\w+)', sql, re.IGNORECASE)
    tables = list(set(table_matches))
    column_matches = re.findall(r'(?:SELECT|WHERE|GROUP BY|HAVING|ORDER BY)\s+([\w\s,\.*]+)', sql, re.IGNORECASE)

    columns = []
    for match in column_matches:
        parts = [p.strip() for p in match.split(',')]
        for part in parts:
            col = re.split(r'\s', part, 1)[0]
            if col and col != '*':
                columns.append(col)

    return tables, columns

tables_dict = defaultdict(list)

for i in range(len(df)):
    query = df.iloc[i]['query']
    tables, columns = extract_tables_and_columns(query)

    for table in tables:
        for column in columns:
            if '.' in column:
                table_part, column_part = column.split('.')
                if table_part == table and column_part not in tables_dict[table]:
                    tables_dict[table].append(column_part)
            elif column not in tables_dict[table]:
                tables_dict[table].append(column)
tables_dict = dict(tables_dict)
tables=tables_dict
tables

{'billing': ['insurance_info',
  'status',
  'visit_id',
  'amount',
  'bill_id',
  'doctor_id',
  'patient_id'],
 'visits': ['patient_id',
  'visit_id',
  'doctor_id',
  'visit_date',
  'diagnosis',
  'treatment'],
 'employees': ['department',
  'role',
  'last_name',
  'employee_id',
  'salary',
  'first_name'],
 'patients': ['phone',
  'ssn',
  'gender',
  'dob',
  'last_name',
  'patient_id',
  'first_name',
  'address',
  'doctor_id']}

# SQL Clause Extraction
Identify the main SQL clause used in each query and add it as a feature.

In [503]:
clause_pattern = re.compile(
    r'(SELECT|FROM|WHERE|GROUP BY|HAVING|ORDER BY|LIMIT|JOIN|INNER JOIN|LEFT JOIN|RIGHT JOIN)\s',
    re.IGNORECASE
)

df['clauses'] = df['query'].apply(
    lambda query: (match.group(1) if (match := clause_pattern.search(query)) else None
))

# Extract Non-Clause Parts
Extract and store non-clause parts of each SQL query for further analysis.

In [504]:
non_clauses = []

for i in range(len(df)):
    query = df.iloc[i]['query']
    last_end = 0
    non_clause_parts = []

    for match in clause_pattern.finditer(query):
        non_clause = query[last_end:match.start()].strip()
        if non_clause:
            filtered = re.sub(r'([^=]*=.*|\d+)', '', non_clause).strip()
            if filtered:
                non_clause_parts.append(filtered)
        last_end = match.end()

    remaining = query[last_end:].strip()
    if remaining:
        filtered = re.sub(r'([^=]*=.*|\d+)', '', remaining).strip()
        if filtered:
            non_clause_parts.append(filtered)

    non_clauses.append(non_clause_parts)

df['non_clauses'] = non_clauses
df['non_clauses']

,non_clauses
0,"[insurance_info, status, visit_id, amount, bil..."
1,"[patient_id, visits]"
2,"[department, role, last_name, employee_id, emp..."
3,"[visits.visit_id, visits.patient_id, visits.do..."
4,"[phone, ssn, patients]"
...,...
995,"[patients.phone, patients.address, patients.fi..."
996,"[patients.patient_id, patients.first_name, pat..."
997,"[patients.patient_id, patients.first_name, pat..."
998,"[phone, address, patients]"


# Create C-Triplets
Form C-Triplets for each query, consisting of the clause, number of tables, and number of columns.

In [505]:
# Create C-Triplets: (clauses, num_tables, num_columns) per query
df['c-triplets'] = list(zip(
    df['clauses'],
    df['tables'].apply(len),
    df['columns'].apply(len)
))
df['c-triplets']

,c-triplets
0,"(SELECT, 1, 4)"
1,"(SELECT, 1, 1)"
2,"(SELECT, 1, 4)"
3,"(SELECT, 2, 7)"
4,"(SELECT, 1, 2)"
...,...
995,"(SELECT, 2, 11)"
996,"(SELECT, 2, 14)"
997,"(SELECT, 2, 14)"
998,"(SELECT, 1, 2)"


# Create M-Triplets
Form M-Triplets for each query, representing table usage and column counts per table.

In [506]:
#Forming M-Triplet - Table Lists
table_lists=[]
for i in range(len(df)):
  table_list=[]
  for table in tables:
    if table in df.iloc[i]['tables']:
      table_list.append(1)
    else:
      table_list.append(0)
  table_lists.append(table_list)

column_counts = []
for query_columns in df['columns']:
    counts = []
    for table in tables:
        count = sum(1 for col in query_columns
                if col.startswith(f"{table}.") or ('.' not in col and col in tables[table]))
        counts.append(count)
    column_counts.append(counts)



# Table List Output
Display the binary table list for each query, indicating which tables are accessed.

In [507]:
table_lists

[[1, 0, 0, 0],
 [0, 1, 0, 0],
 [0, 0, 1, 0],
 [1, 1, 0, 0],
 [0, 0, 0, 1],
 [1, 1, 0, 0],
 [0, 0, 0, 1],
 [1, 1, 0, 0],
 [0, 0, 0, 1],
 [0, 0, 0, 1],
 [0, 0, 0, 1],
 [1, 0, 0, 0],
 [0, 1, 0, 1],
 [1, 0, 0, 0],
 [1, 0, 0, 1],
 [0, 1, 0, 0],
 [0, 0, 0, 1],
 [0, 0, 0, 1],
 [0, 1, 0, 0],
 [0, 0, 0, 1],
 [0, 0, 0, 1],
 [0, 0, 0, 1],
 [1, 0, 0, 0],
 [1, 0, 0, 1],
 [1, 0, 0, 0],
 [1, 0, 0, 0],
 [0, 0, 0, 1],
 [1, 1, 0, 0],
 [0, 0, 1, 0],
 [0, 0, 0, 1],
 [0, 1, 0, 0],
 [0, 0, 0, 1],
 [1, 1, 0, 0],
 [1, 1, 0, 0],
 [1, 1, 0, 0],
 [1, 0, 0, 1],
 [1, 0, 0, 0],
 [1, 0, 0, 0],
 [0, 1, 0, 0],
 [0, 0, 0, 1],
 [0, 1, 0, 0],
 [1, 0, 0, 0],
 [0, 1, 0, 0],
 [1, 0, 0, 1],
 [0, 0, 1, 0],
 [1, 1, 0, 0],
 [0, 1, 0, 0],
 [1, 0, 0, 1],
 [1, 0, 0, 1],
 [0, 0, 1, 0],
 [1, 0, 0, 0],
 [1, 0, 0, 0],
 [0, 0, 0, 1],
 [1, 0, 0, 0],
 [0, 0, 0, 1],
 [1, 0, 0, 0],
 [0, 1, 0, 0],
 [0, 1, 0, 0],
 [1, 0, 0, 0],
 [1, 1, 0, 0],
 [1, 0, 0, 0],
 [0, 1, 0, 0],
 [1, 1, 0, 0],
 [1, 0, 0, 0],
 [0, 1, 0, 0],
 [0, 1, 1, 0],
 [1, 0, 0,

# Column Count Output
Display the count of columns accessed per table for each query.

In [508]:
column_counts

[[4, 1, 0, 0],
 [1, 1, 0, 1],
 [0, 0, 4, 1],
 [1, 6, 0, 0],
 [0, 0, 0, 2],
 [6, 2, 0, 0],
 [0, 0, 1, 3],
 [6, 6, 0, 0],
 [1, 1, 1, 4],
 [1, 1, 2, 4],
 [0, 0, 1, 1],
 [5, 2, 0, 1],
 [0, 6, 0, 8],
 [5, 1, 0, 1],
 [6, 0, 0, 5],
 [1, 2, 0, 1],
 [0, 0, 2, 2],
 [1, 1, 2, 5],
 [3, 6, 0, 2],
 [1, 1, 1, 4],
 [0, 0, 2, 4],
 [1, 1, 1, 2],
 [3, 1, 0, 0],
 [6, 0, 0, 1],
 [3, 1, 0, 0],
 [6, 2, 0, 1],
 [1, 1, 1, 2],
 [6, 6, 0, 0],
 [0, 0, 3, 1],
 [1, 1, 2, 8],
 [1, 1, 0, 1],
 [0, 0, 1, 2],
 [6, 1, 0, 0],
 [6, 2, 0, 0],
 [2, 6, 0, 0],
 [6, 0, 0, 8],
 [6, 2, 0, 1],
 [6, 2, 0, 1],
 [0, 1, 0, 0],
 [0, 0, 1, 4],
 [1, 3, 0, 1],
 [3, 1, 0, 1],
 [2, 3, 0, 1],
 [6, 0, 0, 8],
 [0, 0, 3, 1],
 [6, 3, 0, 0],
 [1, 1, 0, 0],
 [6, 0, 0, 1],
 [6, 0, 0, 8],
 [0, 0, 1, 0],
 [1, 1, 0, 1],
 [5, 1, 0, 1],
 [0, 0, 1, 2],
 [3, 1, 0, 0],
 [0, 0, 0, 2],
 [1, 1, 0, 1],
 [1, 2, 0, 0],
 [1, 1, 0, 0],
 [6, 2, 0, 1],
 [1, 6, 0, 0],
 [5, 2, 0, 1],
 [1, 1, 0, 1],
 [6, 2, 0, 0],
 [5, 2, 0, 1],
 [0, 1, 0, 0],
 [0, 6, 6, 0],
 [6, 2, 0,

# Form M-Triplets
Combine clause, table list, and column count into M-Triplets for each query.

In [509]:
#Forming triplets
df['m-triplets']=list(zip(df['clauses'],table_lists,column_counts))
df['m-triplets']

,m-triplets
0,"(SELECT, [1, 0, 0, 0], [4, 1, 0, 0])"
1,"(SELECT, [0, 1, 0, 0], [1, 1, 0, 1])"
2,"(SELECT, [0, 0, 1, 0], [0, 0, 4, 1])"
3,"(SELECT, [1, 1, 0, 0], [1, 6, 0, 0])"
4,"(SELECT, [0, 0, 0, 1], [0, 0, 0, 2])"
...,...
995,"(SELECT, [1, 0, 0, 1], [6, 0, 0, 5])"
996,"(SELECT, [1, 0, 0, 1], [6, 0, 0, 8])"
997,"(SELECT, [0, 1, 0, 1], [0, 6, 0, 8])"
998,"(SELECT, [0, 0, 0, 1], [0, 0, 0, 2])"


# Create F-Triplets
Form F-Triplets for each query, representing detailed table-column access vectors.

In [510]:
#Formation of F Triplets
column_vectors = []
max_attrs = max(len(attrs) for table in tables.values() for attrs in [table])

for query_columns in df['columns']:
    vector = []
    for table in tables:
        table_vector = []
        for attr in tables[table]:
            is_accessed = any(
                col == f"{table}.{attr}" or
                (('.' not in col) and col == attr and attr in tables[table])
                for col in query_columns
            )
            table_vector.append(1 if is_accessed else 0)
        # Pad with zeros to match max_attrs
        table_vector += [0] * (max_attrs - len(table_vector))
        vector.append(table_vector)
    column_vectors.append(vector)

df['table_column_vectors']=column_vectors
df['f-triplets'] = list(zip(df['clauses'],table_lists,df['table_column_vectors']))
df['f-triplets']

,f-triplets
0,"(SELECT, [1, 0, 0, 0], [[1, 1, 1, 1, 0, 0, 0, ..."
1,"(SELECT, [0, 1, 0, 0], [[0, 0, 0, 0, 0, 0, 1, ..."
2,"(SELECT, [0, 0, 1, 0], [[0, 0, 0, 0, 0, 0, 0, ..."
3,"(SELECT, [1, 1, 0, 0], [[0, 0, 0, 0, 1, 0, 0, ..."
4,"(SELECT, [0, 0, 0, 1], [[0, 0, 0, 0, 0, 0, 0, ..."
...,...
995,"(SELECT, [1, 0, 0, 1], [[1, 1, 1, 1, 1, 0, 1, ..."
996,"(SELECT, [1, 0, 0, 1], [[1, 1, 1, 1, 1, 0, 1, ..."
997,"(SELECT, [0, 1, 0, 1], [[0, 0, 0, 0, 0, 0, 0, ..."
998,"(SELECT, [0, 0, 0, 1], [[0, 0, 0, 0, 0, 0, 0, ..."


# Extract SQL Commands and Features
Extract SQL command features and corresponding numerical features for C, M, and F triplets.

In [511]:
C_SQL_CMD=[data[0] for data in df['c-triplets']]
c_numbers_feat=[(data[1], data[2]) for data in df['c-triplets']]

M_SQL_CMD=[data[0] for data in df['m-triplets']]
m_numbers_feat=[(data[1], data[2]) for data in df['m-triplets']]

F_SQL_CMD=[data[0] for data in df['f-triplets']]
f_numbers_feat=[(data[1], data[2]) for data in df['f-triplets']]

# Vectorize SQL Commands
Vectorize the SQL command features using CountVectorizer for machine learning input.

In [512]:
vectorizer = CountVectorizer(binary=True)

# Replace None with empty string
C_SQL_CMD = [cmd if cmd is not None else "" for cmd in C_SQL_CMD]
M_SQL_CMD = [cmd if cmd is not None else "" for cmd in M_SQL_CMD]
F_SQL_CMD = [cmd if cmd is not None else "" for cmd in F_SQL_CMD]

# Now fit_transform
C_SQL_CMD = vectorizer.fit_transform(C_SQL_CMD)
M_SQL_CMD = vectorizer.transform(M_SQL_CMD)  # Use same vectorizer (no refit)
F_SQL_CMD = vectorizer.transform(F_SQL_CMD)  # Use same vectorizer (no refit)

# View Vectorized C_SQL_CMD Array
Display the vectorized array for C SQL command features.

In [513]:
C_SQL_CMD.toarray()

array([[0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       ...,
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0]])

# View Vectorized M_SQL_CMD Array
Display the vectorized array for M SQL command features.

In [514]:
M_SQL_CMD.toarray()

array([[0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       ...,
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0]])

# View Vectorized F_SQL_CMD Array
Display the vectorized array for F SQL command features.

In [515]:
F_SQL_CMD.toarray()

array([[0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       ...,
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0]])

# Concatenate C and M Features
Concatenate the vectorized SQL command arrays with their respective numerical features for C and M triplets.

In [516]:
m_numbers_feat = np.array(m_numbers_feat)
c_numbers_feat = np.array(c_numbers_feat)

m_numbers_feat_reshaped = m_numbers_feat.reshape(m_numbers_feat.shape[0], -1)

C_SQL_CMD = np.concatenate((C_SQL_CMD.toarray(), c_numbers_feat), axis=1)
M_SQL_CMD = np.concatenate((M_SQL_CMD.toarray(), m_numbers_feat_reshaped), axis=1)

# View c_numbers_feat Array
Display the numerical features for C triplets.

In [517]:
c_numbers_feat

array([[ 1,  4],
       [ 1,  1],
       [ 1,  4],
       ...,
       [ 2, 14],
       [ 1,  2],
       [ 2,  9]])

# View m_numbers_feat Array
Display the numerical features for M triplets.

In [518]:
m_numbers_feat

array([[[1, 0, 0, 0],
        [4, 1, 0, 0]],

       [[0, 1, 0, 0],
        [1, 1, 0, 1]],

       [[0, 0, 1, 0],
        [0, 0, 4, 1]],

       ...,

       [[0, 1, 0, 1],
        [0, 6, 0, 8]],

       [[0, 0, 0, 1],
        [0, 0, 0, 2]],

       [[0, 1, 0, 1],
        [0, 6, 0, 3]]])

# Process and Flatten F_SQL_CMD Features
Process and flatten the F triplet features for use in machine learning models.

In [519]:
F=[]
f=[]
for vectors_i, vectors in f_numbers_feat:
    flat_features = np.array(vectors).flatten()
    F.append(flat_features)
    f.append(vectors_i)

F = np.array(F)
f = np.array(f)
f_numbers_feat=np.concatenate((F,f),axis=1)

# View Flattened f_numbers_feat Array
Display the flattened numerical features for F triplets.

In [520]:
f_numbers_feat

array([[1, 1, 1, ..., 0, 0, 0],
       [0, 0, 0, ..., 1, 0, 0],
       [0, 0, 0, ..., 0, 1, 0],
       ...,
       [0, 0, 0, ..., 1, 0, 1],
       [0, 0, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 1, 0, 1]])

# Concatenate and View Final F_SQL_CMD Feature Array
Concatenate the vectorized F SQL command array with the processed F features and display it.

In [521]:
F_SQL_CMD = np.concatenate((F_SQL_CMD.toarray(),f_numbers_feat), axis=1)
F_SQL_CMD

array([[0, 1, 0, ..., 0, 0, 0],
       [0, 1, 0, ..., 1, 0, 0],
       [0, 1, 0, ..., 0, 1, 0],
       ...,
       [0, 1, 0, ..., 1, 0, 1],
       [0, 1, 0, ..., 0, 0, 1],
       [0, 1, 0, ..., 1, 0, 1]])

# View Final C_SQL_CMD Feature Array
Display the final concatenated C feature array.

In [522]:
C_SQL_CMD

array([[ 0,  1,  0,  1,  4],
       [ 0,  1,  0,  1,  1],
       [ 0,  1,  0,  1,  4],
       ...,
       [ 0,  1,  0,  2, 14],
       [ 0,  1,  0,  1,  2],
       [ 0,  1,  0,  2,  9]])

# View Final M_SQL_CMD Feature Array
Display the final concatenated M feature array.

In [523]:
M_SQL_CMD

array([[0, 1, 0, ..., 1, 0, 0],
       [0, 1, 0, ..., 1, 0, 1],
       [0, 1, 0, ..., 0, 4, 1],
       ...,
       [0, 1, 0, ..., 6, 0, 8],
       [0, 1, 0, ..., 0, 0, 2],
       [0, 1, 0, ..., 6, 0, 3]])

# Train Classifier on C Features
Train a Multinomial Naive Bayes classifier using the C feature set.

In [524]:
C_CLF=MultinomialNB()
CX_train, CX_test, Cy_train, Cy_test = train_test_split(
    C_SQL_CMD, y, test_size=0.2, random_state=42
)
C_CLF.fit(CX_train, Cy_train)

MultinomialNB()

# Evaluate C Classifier
Evaluate the accuracy of the classifier trained on C features.

In [525]:
C_CLF.score(CX_test, Cy_test)

0.42

# Train Classifier on M Features
Train a Multinomial Naive Bayes classifier using the M feature set.

In [526]:
M_CLF=MultinomialNB()
MX_train, MX_test, My_train, My_test = train_test_split(
    M_SQL_CMD, y, test_size=0.2, random_state=42
)
M_CLF.fit(MX_train, My_train)

MultinomialNB()

# Evaluate M Classifier
Evaluate the accuracy of the classifier trained on M features.

In [527]:
M_CLF.score(MX_test, My_test)

0.54

# Train Classifier on F Features
Train a Multinomial Naive Bayes classifier using the F feature set.

In [528]:
F_CLF=MultinomialNB()
FX_train, FX_test, Fy_train, Fy_test = train_test_split(
    F_SQL_CMD, y, test_size=0.2, random_state=42
)
F_CLF.fit(FX_train, Fy_train)

MultinomialNB()

# Evaluate F Classifier
Evaluate the accuracy of the classifier trained on F features.

In [529]:
F_CLF.score(FX_test, Fy_test)

0.7

# Preview DataFrame
Display the first few rows of the DataFrame to inspect the data.

In [530]:
df.head()

,timestamp,user_id,role,query,operation,tables,columns,is_normal,is_join,clauses,non_clauses,c-triplets,m-triplets,table_column_vectors,f-triplets
0,2025-02-24 23:29:13.699838,b249,billing_clerk,"SELECT insurance_info, status, visit_id, amoun...",SELECT,[billing],"[insurance_info, status, visit_id, amount]",True,False,SELECT,"[insurance_info, status, visit_id, amount, bil...","(SELECT, 1, 4)","(SELECT, [1, 0, 0, 0], [4, 1, 0, 0])","[[1, 1, 1, 1, 0, 0, 0, 0, 0], [0, 1, 0, 0, 0, ...","(SELECT, [1, 0, 0, 0], [[1, 1, 1, 1, 0, 0, 0, ..."
1,2024-08-15 22:08:13.728513,d151,doctor,SELECT patient_id FROM visits WHERE doctor_id ...,SELECT,[visits],[patient_id],True,False,SELECT,"[patient_id, visits]","(SELECT, 1, 1)","(SELECT, [0, 1, 0, 0], [1, 1, 0, 1])","[[0, 0, 0, 0, 0, 0, 1, 0, 0], [1, 0, 0, 0, 0, ...","(SELECT, [0, 1, 0, 0], [[0, 0, 0, 0, 0, 0, 1, ..."
2,2025-02-02 03:34:13.686448,a961,admin,"SELECT department, role, last_name, employee_i...",SELECT,[employees],"[department, role, last_name, employee_id]",True,False,SELECT,"[department, role, last_name, employee_id, emp...","(SELECT, 1, 4)","(SELECT, [0, 0, 1, 0], [0, 0, 4, 1])","[[0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, ...","(SELECT, [0, 0, 1, 0], [[0, 0, 0, 0, 0, 0, 0, ..."
3,2024-08-21 23:53:13.720165,d154,doctor,"SELECT visits.visit_id, visits.patient_id, vis...",SELECT,"[visits, billing]","[visits.visit_id, visits.patient_id, visits.do...",True,True,SELECT,"[visits.visit_id, visits.patient_id, visits.do...","(SELECT, 2, 7)","(SELECT, [1, 1, 0, 0], [1, 6, 0, 0])","[[0, 0, 0, 0, 1, 0, 0, 0, 0], [1, 1, 1, 1, 1, ...","(SELECT, [1, 1, 0, 0], [[0, 0, 0, 0, 1, 0, 0, ..."
4,2025-03-10 05:11:13.720634,a480,admin,"SELECT phone, ssn FROM patients WHERE patient_...",SELECT,[patients],"[phone, ssn]",True,False,SELECT,"[phone, ssn, patients]","(SELECT, 1, 2)","(SELECT, [0, 0, 0, 1], [0, 0, 0, 2])","[[0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, ...","(SELECT, [0, 0, 0, 1], [[0, 0, 0, 0, 0, 0, 0, ..."


# Cleanup
Remove the generated database logs file if needed.

In [531]:
# !rm database_logs.json
